## PPO Rollout Knapsack Problem

In [ ]:
from utils.masking_model import ActionMaskModel, ActionMaskModel50, ActionMaskModel100
from utils.custom_knapsack import Knapsack

import ray
from ray.tune.logger import pretty_print
from ray.rllib.models import ModelCatalog
from ray.tune.utils.util import SafeFallbackEncoder
import ray.rllib.agents.ppo as ppo
from ray import tune
import json
import yaml
import numpy as np
import argparse
import matplotlib.pyplot as plt
import pickle
from train_ppo_knapsack_amask import final_evaluation
import matplotlib.pyplot as plt
import seaborn as sns

import or_gym
from or_gym.utils import create_env


In [ ]:
def register_env(env_name, env_config={}):
    env = create_env(env_name)
    tune.register_env(env_name, 
        lambda env_name: env(env_name,
            env_config=env_config))

def get_ppo_trainer(args=None):
    """ Configure the ppo trainer based on training strategy """
    env_name = "Knapsack-v0" # will need to change if using Knapsack-v1
    env_config = {
                'N': 5,
                'max_weight': 15,
                'item_weights': np.array([1, 12, 2, 1, 4]),
                'item_values': np.array([2, 4, 2, 1, 10]),
                'mask': True}
    
    register_env(env_name, env_config) # HAVE TO REGISTER KNAPSACK ENV
    config = ppo.DEFAULT_CONFIG.copy()
    config["env"] = Knapsack
    config["env_config"] = env_config
    
    if args:
        # add stuff for different size maps
        if args['items'] == 50:
            env_config = {
                'N': 50,
                'max_weight': 75,
                'item_weights': np.array([40,  7, 32, 26, 13, 31, 43, 13, 11, 28, 
                                          9, 48, 42, 28, 45, 17, 45, 46, 26,  6, 22, 
                                          8, 18, 29, 42, 49, 27, 14, 46, 34, 33, 32, 
                                          18, 14, 14, 48, 32, 24, 40, 22, 19, 37, 37,
                                          18, 37, 29, 11, 35, 37, 11]),
                'item_values': np.array([17, 10,  7, 11, 10,  9,  2, 15,  8,  8,  7,
                                         12,    9, 12,  4, 11, 18, 17,  5,  5,  4,  
                                         3, 10, 13,  9, 14, 12, 11,  1, 14, 13,  6, 
                                         16,  8, 13, 14, 19, 18, 17,  3, 16,  3, 13,  
                                         2, 10, 16, 14,  4, 11, 14]),
                'mask': True}
            config["env_config"].update(env_config)
            
        elif args['items'] == 100:
            env_config = {
                'N': 100,
                'max_weight': 50,
                'item_weights': np.array([33, 21, 29, 17, 33, 32, 35, 23, 22, 
                                          18, 49, 47,  8, 41, 49, 45, 29, 49, 43, 49, 15,  
                                          7, 46, 47, 18, 46, 20, 38, 38, 30, 37, 39, 30, 48, 
                                          35, 19, 16, 31, 26, 41, 21,  7, 48, 41, 28, 37, 11, 
                                          6,  9, 33, 30, 12, 14, 25, 40, 22, 33, 33, 24, 16, 
                                          15, 12,  6, 49, 33, 35, 33, 39, 5, 34,  6, 43, 48, 
                                          25, 33, 11, 31, 46, 26, 26, 20, 47, 46, 46, 46, 14,
                                          37, 27, 40, 12, 21, 45, 21, 21, 15, 38, 10, 37, 21, 27]),
                'item_values': np.array([ 6, 14,  6, 14,  7,  5, 14,  8, 14, 11, 12, 12, 
                                         12, 14, 10,  2, 10,3,  6,  1,  3,  2,  8, 10,  2, 
                                         12,  4,  2,  4,  8, 14,  3,  6,  8, 4,  2, 14,  8,  
                                         7, 13,  5,  2,  5, 12,  2,  1, 11, 11,  2,  8,  5, 
                                         9,  5,  2, 13,  9, 12,  7,  7,  8,  7, 11,  4,  2,  
                                         4,  5,  5, 13, 4,  6, 13,  8,  5,  4,  2, 11, 11, 
                                         10, 13,  7,  7,  5, 14,  3,  2, 14, 14,  5, 11, 12, 
                                         12,  6,  6, 12,  5,  2, 10,  5,  8, 14]),
                'mask': True}
            config["env_config"].update(env_config)
            
        if args["strategy"] == "action_masking":
            print('\nUsing action masking to train ...\n')
            if args["items"] == 5:
                ModelCatalog.register_custom_model("kp_mask", ActionMaskModel)
                config["model"] = {
                    "custom_model": "kp_mask",
                }
                
            elif args["items"] == 50:
                ModelCatalog.register_custom_model("kp_mask", ActionMaskModel50)
                config["model"] = {
                    "custom_model": "kp_mask",
                }
                
            elif args["items"] == 100:
                ModelCatalog.register_custom_model("kp_mask", ActionMaskModel100)
                config["model"] = {
                    "custom_model": "kp_mask",
                }
                
            config["env_config"].update({"mask":True})
        elif args["strategy"] == "runtime":
            config["env_config"].update({"runtime":True})
            
        config["env_config"]["seed"] = args["seed"]
        config.update({'seed': args["seed"]})
        
    trainer = ppo.PPOTrainer(config=config)
    return trainer, config["env_config"]

## 5 Item SRL-AM

In [ ]:
method = "ppo"

args = {
    "items": 5,
    "num_rollouts": 1,
    "method": "ppo",
    "strategy": "action_masking",
    "seed": 4,
}

optimal_solution = {
            5: 36,
            50: 104,
            100: 104,
}

mask_agent, mask_env_config = get_ppo_trainer(args)
name = "knapsack_ppo_masking_seed-{}_checkpoint-{}".format(args["seed"], str(args["items"]))
mask_agent.restore("trained_agents/{}/{}".format(name, name))
mask_eval_reward, mask_eval_time, mask_v_total, mask_v_eps, path = final_evaluation(mask_agent, args["num_rollouts"], env_config=mask_env_config)


# print("\n----- Demo With Masking -----")
# print("Avg. Rollout Reward WITH AM: ", mask_eval_reward)
# print("Avg. Num of Steps WITH AM: ", mask_eval_time)
# print("Total Violations WITH AM: ", mask_v_total)
# print("Percentage of Safe Rollouts WITH AM: {}%".format(100-(mask_v_eps/args["num_rollouts"]*100)))

plt.close()
sns.set()
csfont = {'fontname':'Times New Roman',  'fontsize':18}
# lfont = {'fontname':'Times New Roman', 'fontsize':12}
plt.figure(1)
plt.plot(len(path["val"]), optimal_solution[args["items"]], 'o', markersize=18, color="#00FF00", label="Optimal Solution")
plt.plot(range(1, len(path["val"]) + 1), path["val_sum"], '-o', color='green', label="Total Value")
plt.plot(range(1, len(path["val"]) + 1), [mask_env_config["max_weight"]]*len(path["val"]), '--', linewidth=3, color='red', label="Max Weight")
plt.bar(range(1, len(path["lb_sum"]) + 1), path["lb_sum"], color='blue', label="Total Weight")
plt.bar(range(1, len(path["lb"]) +1), path["lb"], color="orange", edgecolor="black", hatch="//", label="Chosen Item Weight")
plt.legend(fontsize=12)
# plt.title("SRL Rollout in the {} Item Knapsack Environment".format(str(args["items"])), **csfont)
plt.xticks(**csfont) 
plt.yticks(**csfont)
plt.ylabel("Value", **csfont)
plt.xlabel("Step", **csfont)
plt.tight_layout()
# plt.savefig("images/knapsack_{}_am_{}_results.png".format(args["method"], str(args["items"])), bbox_inches='tight', dpi=200)
plt.show()


## 50 Item SRL-AM

In [ ]:
method = "ppo"

args = {
    "items": 50,
    "num_rollouts": 1,
    "method": "ppo",
    "strategy": "action_masking",
    "seed": 4,
}

optimal_solution = {
            5: 36,
            50: 104,
            100: 104,
}

mask_agent, mask_env_config = get_ppo_trainer(args)
name = "knapsack_ppo_masking_seed-{}_checkpoint-{}".format(args["seed"], str(args["items"]))
mask_agent.restore("trained_agents/{}/{}".format(name, name))
mask_eval_reward, mask_eval_time, mask_v_total, mask_v_eps, path = final_evaluation(mask_agent, args["num_rollouts"], env_config=mask_env_config)


# print("\n----- Demo With Masking -----")
# print("Avg. Rollout Reward WITH AM: ", mask_eval_reward)
# print("Avg. Num of Steps WITH AM: ", mask_eval_time)
# print("Total Violations WITH AM: ", mask_v_total)
# print("Percentage of Safe Rollouts WITH AM: {}%".format(100-(mask_v_eps/args["num_rollouts"]*100)))

plt.close()
sns.set()
csfont = {'fontname':'Times New Roman',  'fontsize':18}
# lfont = {'fontname':'Times New Roman', 'fontsize':12}
plt.figure(1)
plt.plot(len(path["val"])-1, optimal_solution[args["items"]], 'o', markersize=18, color="#00FF00", label="Optimal Solution")
plt.plot(range(1, len(path["val"])), path["val_sum"][:-1], '-o', color='green', label="Total Value")
plt.plot(range(1, len(path["val"])), [mask_env_config["max_weight"]]*(len(path["val"])-1), '--', linewidth=3, color='red', label="Max Weight")
plt.bar(range(1, len(path["lb_sum"])), path["lb_sum"][:-1], color='blue', label="Total Weight")
plt.bar(range(1, len(path["lb"])), path["lb"][:-1], color="orange", edgecolor="black", hatch="//", label="Chosen Item Weight")
plt.legend(fontsize=12)
# plt.title("SRL Rollout in the {} Item Knapsack Environment".format(str(items)), **csfont)
plt.xticks(**csfont) 
plt.yticks(**csfont)
plt.ylabel("Value", **csfont)
plt.xlabel("Step", **csfont)
plt.tight_layout()
# plt.savefig("images/knapsack_{}_am_{}_results.png".format(args["method"], str(args["items"])), bbox_inches='tight', dpi=200)
plt.show()


## 100 Item SRL-AM

In [ ]:
method = "ppo"

args = {
    "items": 100,
    "num_rollouts": 1,
    "method": "ppo",
    "strategy": "action_masking",
    "seed": 4,
}

optimal_solution = {
            5: 36,
            50: 104,
            100: 104,
}

mask_agent, mask_env_config = get_ppo_trainer(args)
name = "knapsack_ppo_masking_seed-{}_checkpoint-{}".format(args["seed"], str(args["items"]))
mask_agent.restore("trained_agents/{}/{}".format(name, name))
mask_eval_reward, mask_eval_time, mask_v_total, mask_v_eps, path = final_evaluation(mask_agent, args["num_rollouts"], env_config=mask_env_config)


# print("\n----- Demo With Masking -----")
# print("Avg. Rollout Reward WITH AM: ", mask_eval_reward)
# print("Avg. Num of Steps WITH AM: ", mask_eval_time)
# print("Total Violations WITH AM: ", mask_v_total)
# print("Percentage of Safe Rollouts WITH AM: {}%".format(100-(mask_v_eps/args["num_rollouts"]*100)))

plt.close()
sns.set()
csfont = {'fontname':'Times New Roman',  'fontsize':18}
# lfont = {'fontname':'Times New Roman', 'fontsize':12}
plt.figure(1)
plt.plot(len(path["val"])-1, optimal_solution[args["items"]], 'o', markersize=18, color="#00FF00", label="Optimal Solution")
plt.plot(range(1, len(path["val"])), path["val_sum"][:-1], '-o', color='green', label="Total Value")
plt.plot(range(1, len(path["val"])), [mask_env_config["max_weight"]]*(len(path["val"])-1), '--', linewidth=3, color='red', label="Max Weight")
plt.bar(range(1, len(path["lb_sum"])), path["lb_sum"][:-1], color='blue', label="Total Weight")
plt.bar(range(1, len(path["lb"])), path["lb"][:-1], color="orange", edgecolor="black", hatch="//", label="Chosen Item Weight")
plt.legend(fontsize=12)
# plt.title("SRL Rollout in the {} Item Knapsack Environment".format(str(items)), **csfont)
plt.xticks(**csfont) 
plt.yticks(**csfont)
plt.ylabel("Value", **csfont)
plt.xlabel("Step", **csfont)
plt.tight_layout()
# plt.savefig("images/knapsack_{}_am_{}_results.png".format(args["method"], str(args["items"])), bbox_inches='tight', dpi=200)
plt.show()
